<a href="https://colab.research.google.com/github/bee-llel/Machine-Learning/blob/master/FeedForwardNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#librairy:
import pandas as pd # used to manipulate the data 
import numpy as np
import scipy as sp 

In [3]:
#loading the data in a dataframe

data = pd.read_csv('data_ffnn_3classes.txt', sep=" ", header=None)

In [13]:
data.columns = ['feature1','feature2','target']
data.head(10)

,feature1,feature2,target
0,1.9643,4.5957,1.0
1,2.2753,3.8589,1.0
2,2.9781,4.5651,1.0
3,2.9320,3.5519,1.0
4,3.5772,2.8560,1.0


In [29]:
#variable definition: first we will take the exercice values then we will generalize our function once it is working

##############################
#
#N = number of feature = 2
#
#I = data size = 71
#
#J = output dimension = 3
#
#K = number of hidden neuron = how many we want

##############################
#feature matrix:
X = np.transpose(data[['feature1','feature2']].to_numpy())


##############################
#target matrix:
y = data['target'].to_numpy()

#############################
#bias matrix: which seem to be a column of 1 in our simple neural network
B = np.ones(np.transpose(X).shape)

#############################
# W and V matrix: we will initialize them to one until i know what exactly they are

W = np.ones()



In [31]:
X.shape

(2, 71)

In [ ]:
#computation function

In [ ]:
#forward propagation function:

In [ ]:
#backpropagation function:

In [ ]:
#FFNN function:

In [ ]:
#data visualisation:

In [ ]:
#result: